# 04 — Transaction Categorization Test

**Objective:** Execute the transaction categorization using Databricks `ai_query()` against the three test layers (Obvious, Ambiguous, Unknown).

### Tasks:
1. Define the system prompt with full taxonomy context and few-shot examples.
2. Run `ai_query()` on Layer 1 codes (~40 codes).
3. Run `ai_query()` on Layer 2 codes (~11 codes, including EFHDS1).
4. Run `ai_query()` on Layer 3 codes (7 unknown codes).
5. Save all results to the `results/` directory.

In [ ]:
# Configuration
MODEL_NAME = "databricks-meta-llama-3-1-70b-instruct"  # Update based on Step 03 discovery
CATALOG_TABLE = "transaction_code_catalog"           # Name of the table in Unity Catalog
TAXONOMY_PATH = "../taxonomy/transaction_categorization_taxonomy.md"

print(f"Using model: {MODEL_NAME}")

In [ ]:
# 1. Load Taxonomy and Build Prompt
with open(TAXONOMY_PATH, "r") as f:
    taxonomy_md = f.read()

system_prompt = f"""
You are a transaction categorization engine for a US bank. 
Given a transaction code (TRANCD) and description (DESC), classify it into the StrategyCorp taxonomy below.

{taxonomy_md}

### Rules:
1. First determine Block A (Non-fee) or Block B (Fee item). Fee items typically contain: "fee", "charge", "surcharge", "penalty", "reversal".
2. Classify through Level 2 > Level 3 > Level 4.
3. Use "Unclassified" if no mapping fits. Do not guess.
4. Return ONLY valid JSON with keys: category_1, category_2, category_3, category_4, include_in_scoring, credit_debit, confidence.
"""

few_shot_examples = """
### Few-Shot Examples:
Input: TRANCD=183, DESC="ACH Debit - SERMONS"
Output: {"category_1":"Non-fee item","category_2":"Money movement","category_3":"ACH","category_4":null,"include_in_scoring":true,"credit_debit":"Debit","confidence":0.99}

Input: TRANCD=299, DESC="ATM Service Charge"
Output: {"category_1":"Fee item","category_2":"All others","category_3":"Money movement","category_4":"ATM","include_in_scoring":false,"credit_debit":"Debit","confidence":0.98}

Input: TRANCD=333, DESC="Account Service Fee"
Output: {"category_1":"Fee item","category_2":"Service Charges","category_3":null,"category_4":null,"include_in_scoring":false,"credit_debit":"Debit","confidence":0.99}

Input: TRANCD=227, DESC="ATM Withdrawal 0954 01/23/26"
Output: {"category_1":"Non-fee item","category_2":"Money movement","category_3":"ATM","category_4":null,"include_in_scoring":true,"credit_debit":"Debit","confidence":0.98}
"""

full_system_prompt = system_prompt + "\n" + few_shot_examples
print("Prompt prepared.")

### 2. Execute Classification (SQL)
Run this in Databricks to classify all codes in the catalog. We'll pass the `sample_desc_1` as the primary description context.

In [ ]:
classification_query = f"""
WITH raw_results AS (
  SELECT 
    TRANCD,
    sample_desc_1,
    volume,
    source_file,
    ai_query(
      '{MODEL_NAME}',
      CONCAT('{full_system_prompt}', '\nClassify: TRANCD=', TRANCD, ', DESC="', sample_desc_1, '"')
    ) as llm_raw
  FROM {CATALOG_TABLE}
)
SELECT 
  *, 
  from_json(llm_raw, 'category_1 STRING, category_2 STRING, category_3 STRING, category_4 STRING, include_in_scoring BOOLEAN, credit_debit STRING, confidence DOUBLE') as parsed
FROM raw_results
"""

try:
    results_df = spark.sql(classification_query)
    display(results_df)
    
    # Save results for evaluation
    # results_df.write.mode("overwrite").saveAsTable("transaction_classification_results")
    # results_df.coalesce(1).write.format("csv").option("header", "true").save("../results/latest_run.csv")
except NameError:
    print("Spark session not found. Run this in a Databricks notebook cell.")

### 3. Layered Analysis
Optionally run classification separately for each layer to monitor progress.